In [55]:
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

from torch.hub import load_state_dict_from_url
import torch
import torch.nn as nn

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None, pool=True, block_stride=[2, 2, 2, 2]):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.pooling = nn.MaxPool2d(kernel_size=3, stride=block_stride[0], padding=1) if pool==True else Identity()
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=block_stride[1],
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=block_stride[2],
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=block_stride[3],
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pooling(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        print("Final shape : ", x.shape)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x
    
def _resnet(block, layers, **kwargs):
    model = ResNet(block, layers, **kwargs)
    return model



# TODO : Build smaller resnets for 2x, 4x and 8x downsampled inputs.

def resnet18_LR(scale, pretrained=False, progress=True):
    if scale == 2:
        model = _resnet(BasicBlock, [2,2,2,2], pool=False, block_stride=[2,2,2,2])
    elif scale == 4:
        model = _resnet(BasicBlock, [2,2,2,2], pool=False, block_stride=[2,2,2,1])
    elif scale == 8:
        model = _resnet(BasicBlock, [2,2,2,2], pool=False, block_stride=[2,2,1,1])
    elif scale == 16:
        model = _resnet(BasicBlock, [2,2,2,2], pool=False, block_stride=[2,1,1,1])
    else:
        assert "Scale not supported !"
        
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['resnet18'],
                                              progress=progress)
        model.load_state_dict(state_dict)
        
    return model

def resnet34_LR(scale, pretrained=False, progress=True):
    if scale == 2:
        model = _resnet(BasicBlock, [3,4,6,3], pool=False, block_stride=[2,2,2,2])
    elif scale == 4:
        model = _resnet(BasicBlock, [3,4,6,3], pool=False, block_stride=[2,2,2,1])
    elif scale == 8:
        model = _resnet(BasicBlock, [3,4,6,3], pool=False, block_stride=[2,2,1,1])
    elif scale == 16:
        model = _resnet(BasicBlock, [3,4,6,3], pool=False, block_stride=[2,1,1,1])
    else:
        assert "Scale not supported !"
        
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['resnet34'],
                                              progress=progress)
        model.load_state_dict(state_dict)
        
    return model

def resnet50_LR(scale, pretrained=False, progress=True):
    if scale == 2:
        model = _resnet(Bottleneck, [3,4,6,3], pool=False, block_stride=[2,2,2,2])
    elif scale == 4:
        model = _resnet(Bottleneck, [3,4,6,3], pool=False, block_stride=[2,2,2,1])
    elif scale == 8:
        model = _resnet(Bottleneck, [3,4,6,3], pool=False, block_stride=[2,2,1,1])
    elif scale == 16:
        model = _resnet(Bottleneck, [3,4,6,3], pool=False, block_stride=[2,1,1,1])
    else:
        assert "Scale not supported !"
        
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['resnet50'],
                                              progress=progress)
        model.load_state_dict(state_dict)
        
    return model

def resnet101_LR(scale, pretrained=False, progress=True):
    if scale == 2:
        model = _resnet(Bottleneck, [3, 4, 23, 3], pool=False, block_stride=[2,2,2,2])
    elif scale == 4:
        model = _resnet(Bottleneck, [3, 4, 23, 3], pool=False, block_stride=[2,2,2,1])
    elif scale == 8:
        model = _resnet(Bottleneck, [3, 4, 23, 3], pool=False, block_stride=[2,2,1,1])
    elif scale == 16:
        model = _resnet(Bottleneck, [3, 4, 23, 3], pool=False, block_stride=[2,1,1,1])
    else:
        assert "Scale not supported !"
        
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['resnet101'],
                                              progress=progress)
        model.load_state_dict(state_dict)
        
    return model

# Two ways for building smaller LR ResNets
1. Stride modification
2. Block ommision

In [56]:
# Stride Modification

resnet_x2 = resnet18_LR(2, pretrained=True)
resnet_x4 = resnet18_LR(4, pretrained=True)
resnet_x8 = resnet18_LR(8, pretrained=True)
resnet_x16 = resnet18_LR(16, pretrained=True)

x2 = torch.randn([1,3,112,112])
x4 = torch.randn([1,3,56,56])
x8 = torch.randn([1,3,28,28])
x16 = torch.randn([1,3,14,14])

print(resnet_x2(x2).shape)
print(resnet_x4(x4).shape)
print(resnet_x8(x8).shape)
print(resnet_x16(x16).shape)

Final shape :  torch.Size([1, 512, 7, 7])
torch.Size([1, 1000])
Final shape :  torch.Size([1, 512, 7, 7])
torch.Size([1, 1000])
Final shape :  torch.Size([1, 512, 7, 7])
torch.Size([1, 1000])
Final shape :  torch.Size([1, 512, 7, 7])
torch.Size([1, 1000])
